In [62]:
import numpy as np
import pandas as pd
import plotly
from plotly.graph_objs import Figure, Histogram, Layout, Scatter, Bar, Pie
#import scipy as stats
#import seaborn as sns
#plotly.offline.init_notebook_mode(connected=True)
from bs4 import BeautifulSoup as bsoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException

from time import sleep
import re
import requests as rq
import csv
import os
import time

In [98]:
def parse_html(html):
    data, item = pd.DataFrame(), {}
    soup = bsoup(html, 'lxml')
    for i, resto in enumerate(soup.find_all('div', class_='rest-row-info')):
        item['name'] = resto.find('span', class_='rest-row-name-text').text

        booking = resto.find('div', class_='booking')
        item['bookings'] = re.search('\d+', booking.text).group() if booking else 'NA'

        rating = resto.select('.star-rating .star-rating-score')
        #print(rating)
        item['rating'] = rating[0]['aria-label'] if rating else 'NA'

        reviews = resto.find('span', class_='star-rating-text--review-text')
        
        reviews = resto.select('div.review-rating-text span')
        #print(reviews)
        item['reviews'] = reviews[0].text if reviews else 'NA'

        item['price'] = int(resto.find('div', class_='rest-row-pricing').find('i').text.count('$'))
        
        item['cuisine'] = resto.find_all('span', class_='rest-row-meta--cuisine')[-1].text
        #print(item['cuisine'])
        
        item['location'] = resto.find('span', class_='rest-row-meta--location').text
        data[i] = pd.Series(item)
    return data.T


In [118]:
restaurants = pd.DataFrame()
last_page=100
i = 1

options = Options()
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
driver = webdriver.Chrome('./chromedriver',options=options)
driver.get('https://www.opentable.com/los-angeles-restaurant-listings?')

#first data scrape
new_data = parse_html(driver.page_source)
restaurants = pd.concat([restaurants, new_data])
while i<=last_page:
    try:
        next_page_btn = driver.find_element_by_link_text('Next')
        WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.LINK_TEXT,'Next'))).click()
        #subsequent data scrape
        new_data = parse_html(driver.page_source)
        if new_data.empty:
            break
        else:
            restaurants = pd.concat([restaurants, new_data], ignore_index=True)
            print(len(restaurants)) 
            time.sleep(6)
    except:
        print("No more pages left")
        break
    
    i+=1
driver.close()

200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
No more pages left


In [123]:
restaurants

,name,bookings,rating,reviews,price,cuisine,location
0,Rossoblu,66,4.7 stars out of 5,Exceptional,3,Italian,Downtown
1,HATCH Yakitori + Bar,34,4.5 stars out of 5,Awesome,3,Japanese,Downtown
2,Takami Sushi & Robata Restaurant,82,4.5 stars out of 5,Awesome,3,Japanese,Downtown
3,BRERA Ristorante,46,4.7 stars out of 5,Exceptional,3,Italian,Downtown
4,Perch LA,243,4.7 stars out of 5,Exceptional,3,French American,Downtown
...,...,...,...,...,...,...,...
9995,Thorny's Sports Bar & Grill,NA,NA,NA,2,Grill,Highland
9996,Las Conchitas Mexican Restaurant,NA,NA,NA,2,Mexican,San Bernardino
9997,Jovi's Diner,NA,NA,NA,2,American,San Bernardino
9998,The Rustik Fork,NA,NA,NA,2,Farm-to-table,Riverside


In [130]:
np.unique(restaurants.price)

array([2, 3, 4], dtype=object)

In [143]:
priceMap = {2:'<30',3:'31-50',4:'50+'}
restaurants['dollars_price'] = restaurants['price'].map(priceMap)

In [144]:
#restaurants.pop('dollars_price')
restaurants

,name,bookings,rating,reviews,price,cuisine,location,dollars_price
0,Rossoblu,66,4.7 stars out of 5,Exceptional,3,Italian,Downtown,31-50
1,HATCH Yakitori + Bar,34,4.5 stars out of 5,Awesome,3,Japanese,Downtown,31-50
2,Takami Sushi & Robata Restaurant,82,4.5 stars out of 5,Awesome,3,Japanese,Downtown,31-50
3,BRERA Ristorante,46,4.7 stars out of 5,Exceptional,3,Italian,Downtown,31-50
4,Perch LA,243,4.7 stars out of 5,Exceptional,3,French American,Downtown,31-50
...,...,...,...,...,...,...,...,...
9995,Thorny's Sports Bar & Grill,NA,NA,NA,2,Grill,Highland,<30
9996,Las Conchitas Mexican Restaurant,NA,NA,NA,2,Mexican,San Bernardino,<30
9997,Jovi's Diner,NA,NA,NA,2,American,San Bernardino,<30
9998,The Rustik Fork,NA,NA,NA,2,Farm-to-table,Riverside,<30


In [145]:
#output results to csv file
restaurants.to_csv('results2.csv',index=False)

In [21]:
#np.unique(restaurants['name'],return_counts=True)